In [1]:
import numpy as np
import pandas as pd
import ast
df = pd.read_csv('./resume_dataset_1.csv')


In [2]:
#Remove pucntuation in Resume
import string
new_resume_list = []
for resume in df['Resume']:
    resume = ast.literal_eval(resume)
    new_r = resume.translate(str.maketrans('', '', string.punctuation))
    new_resume_list.append(new_r[1:])

new_resume = np.asarray(new_resume_list)
df['Normalized_Resume'] = new_resume
df = df.drop(columns = ['Resume'])
df

TypeError: a bytes-like object is required, not 'dict'

# Extract text

In [ ]:
import PyPDF2
import textract

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def pdf2txt(file_path: str) -> str:
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        texts = ''
        np = pdf_reader.getNumPages()
        for i in range(np):
            texts += pdf_reader.getPage(i).extractText()
    return texts

# Model Selection

In [ ]:
#Calculate Term Frequency, Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf = True, min_df =5, norm = 'l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.Normalized_Resume).toarray()
labels = df.ID
features.shape

In [ ]:
from sklearn.feature_selection import chi2

N = 2
category_id_df = df[['Category','ID']].drop_duplicates().sort_values('ID')
category_to_id = dict(category_id_df.values)
for Category, ID in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == ID)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(Category))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

CV = 5
kf = KFold(n_splits = 5)
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    print(model_name)
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=kf)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer


# X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category'], random_state = 0)
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
cv_df.groupby('model_name').accuracy.mean()